The aim of the script is to polygonize tif files with areas with a slope of less than 15%. For this purpose, a treshold value below 15 percent was calculated. Then coordinates were generated which were base for polygonization of the isolated areas into the shp files.

In [ ]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
from osgeo import osr, ogr
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gdal

In [ ]:
from osgeo import gdal

In [ ]:
!pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 54.6 MB/s eta 0:00:00


In [ ]:
import rasterio
from rasterio.transform import from_origin
from rasterio.warp import calculate_default_transform, reproject
from rasterio.plot import show
from rasterio.features import shapes
from shapely.geometry import shape

In [ ]:
os.chdir("/content/drive/MyDrive/Dane z Kaggla/Elevation/Elevation_input")
!ls

amaska2.tif			     n50_e020_1arc_v3.tif
aspect.tif			     polyg_maska_slopePer_n50_e016_1arc_v3shp
hill_n50_e016_1arc_v3.tif	     polyg_maska_slopePer_n50_e016_1arc_v3.tif
hill_n50_e019_1arc_v3.tif	     polyg_maska_slopePer_n50_e019_1arc_v3shp
hill_n50_e020_1arc_v3.tif	     polyg_maska_slopePer_n50_e019_1arc_v3.tif
hill.tif			     polyg_maska_slopePer_n50_e020_1arc_v3shp
maska_slopePer_n50_e016_1arc_v3.tif  polyg_maska_slopePer_n50_e020_1arc_v3.tif
maska_slopePer_n50_e019_1arc_v3.tif  slopePer_n50_e016_1arc_v3.tif
maska_slopePer_n50_e020_1arc_v3.tif  slopePer_n50_e019_1arc_v3.tif
n50_e016_1arc_v3.tif		     slopePer_n50_e020_1arc_v3.tif
n50_e019_1arc_v3.tif		     slope.tif


Input for calculating treshold (masks)

In [ ]:
src_Path = '/content/drive/MyDrive/Dane z Kaggla/Elevation/Elevation_input'
input_files_slope = [r for r in os.listdir(src_Path) if r.startswith('slopePer')]
input_files_slope

['slopePer_n50_e016_1arc_v3.tif',
 'slopePer_n50_e019_1arc_v3.tif',
 'slopePer_n50_e020_1arc_v3.tif']

In [ ]:
output_masks = []
for srcRst in input_files_slope:
  dstRst = 'maska_' + srcRst[:-4] +srcRst[-4:]
  output_masks.append(dstRst)
output_masks

['maska_slopePer_n50_e016_1arc_v3.tif',
 'maska_slopePer_n50_e019_1arc_v3.tif',
 'maska_slopePer_n50_e020_1arc_v3.tif']

Output: Polygonized shp files

In [ ]:
output_files_polygonized = []
for srcRst in input_files_mask:
  dstRst = 'polyg_' + srcRst[:-4] +'shp'
  output_files_polygonized.append(dstRst)
output_files_polygonized

['polyg_maska_slopePer_n50_e016_1arc_v3shp',
 'polyg_maska_slopePer_n50_e019_1arc_v3shp',
 'polyg_maska_slopePer_n50_e020_1arc_v3shp']

In [ ]:
class Polygonize:
    def __init__(self, input_files_slope, threshold=1600000):
        self.input_files_slope = input_files_slope
        self.threshold = threshold
        self.masks = []
        self.datasets = []

    def mask_slope_below_threshold(self):
        for slope in self.input_files_slope:
            # Open the DEM dataset
            slope_dataset = gdal.Open(slope)

            # Get the DEM raster band
            dem_band = slope_dataset.GetRasterBand(1)

            # Read the DEM as an array
            slope_array = dem_band.ReadAsArray()

            # Create a mask where the slope values are below the threshold
            mask = slope_array < self.threshold

            # Append the mask and dataset to the lists
            self.masks.append(mask)
            self.datasets.append(slope_dataset)

    def save_mask_to_file(self, output_masks):
        for mask, dataset, output_mask in zip(self.masks, self.datasets, output_masks):
            driver = gdal.GetDriverByName('GTiff')
            mask_dataset = driver.Create(output_mask, dataset.RasterXSize, dataset.RasterYSize, 1, gdal.GDT_Byte)
            mask_dataset.SetGeoTransform(dataset.GetGeoTransform())
            mask_dataset.SetProjection(dataset.GetProjection())
            mask_band = mask_dataset.GetRasterBand(1)
            mask_band.WriteArray(mask.astype(np.uint8))
            mask_band.FlushCache()
            mask_band.SetNoDataValue(0)

    def generator_geometry(self, input_file, mask):
        with rasterio.Env():
            with rasterio.open(input_file) as src:
                image = src.read(1)  # first band
                results = (
                    {
                        'properties': {'raster_val': v},
                        'geometry': s
                    }
                    for i, (s, v) in enumerate(
                        shapes(image, mask=mask, transform=src.transform))
                )
                return results

    def polygonize(self, input_file, output_file, mask):
        coordinates = self.generator_geometry(input_file, mask)
        geometry = list(coordinates)
        polygonized_raster = gpd.GeoDataFrame.from_features(geometry)
        polygonized_raster.to_file(output_file, driver='ESRI Shapefile', crs="EPSG:4326")

    def process_files(self, input_files_mask, output_files_polygonized):
        for input_file, output_file, mask in zip(input_files_mask, output_files_polygonized, self.masks):
            self.polygonize(input_file, output_file, mask)
            print(f"Polygonized raster {input_file} saved to {output_file}")

In [ ]:
input_files_slope = input_files_slope
input_files_mask = output_masks
output_files_polygonized = output_files_polygonized



In [ ]:
polygonizer = Polygonize(input_files_slope)
polygonizer.mask_slope_below_threshold()
polygonizer.save_mask_to_file(input_files_mask)
polygonizer.process_files(input_files_mask, output_files_polygonized)

Polygonized raster maska_slopePer_n50_e016_1arc_v3.tif saved to polyg_maska_slopePer_n50_e016_1arc_v3shp
Polygonized raster maska_slopePer_n50_e019_1arc_v3.tif saved to polyg_maska_slopePer_n50_e019_1arc_v3shp
Polygonized raster maska_slopePer_n50_e020_1arc_v3.tif saved to polyg_maska_slopePer_n50_e020_1arc_v3shp
